### Function Calling
#### Enables model to fetch data and takes action

Function calling provides a powerful and flexible way for OpenAI models to interface with your code or external services, and has two primary use cases:

1. Fetching Data: Retrieve up-to-date information to incorporate into the model's response (RAG). Useful for searching knowledge bases and retrieving specific data from APIs (e.g. current weather data).

2. Taking Action: Perform actions like submitting a form, calling APIs, modifying application state (UI/frontend or backend), or taking agentic workflow actions (like handing off the conversation).

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(dotenv_path="../.env")

open_api_key = os.getenv("open_api_key")

openAI_params = {
    'api_key': open_api_key
}
client = OpenAI(**openAI_params)

In [3]:
## Function calling example with get_weather function
tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature for a given location.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "City and country e.g. Bogotá, Colombia"
                }
            },
            "required": [
                "location"
            ],
            "additionalProperties": False
        },
        "strict": True
    }
}]

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": "What is the weather like in Paris today?"}],
    tools=tools
)

print("-"*30, "Details about response", "-"*30, sep='\n')
print(f"Model: {completion.model}")

print("\n", "-"*30, "Details about Results", "-"*30, sep='\n')
print(f"Content Result: {completion.choices[0].message.tool_calls}")


print("\n", "-"*30, "Details about Usage", "-"*30, sep='\n')
print(f"Result: {completion.usage.to_dict()}")

------------------------------
Details about response
------------------------------
Model: gpt-4o-2024-08-06


------------------------------
Details about Results
------------------------------
Content Result: [ChatCompletionMessageToolCall(id='call_WTMS1AsnllTe0eO8DICCAZ8F', function=Function(arguments='{"location":"Paris, France"}', name='get_weather'), type='function')]


------------------------------
Details about Usage
------------------------------
Result: {'completion_tokens': 17, 'prompt_tokens': 65, 'total_tokens': 82, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}


#### Tools can have two forms:
1. Function Calling: Developer Defined code
2. Hosted Tools: OpenAI Built-In Tools

#### Function calling steps

##### Step 1: Call model using system defined along with System and User Message 

In [24]:
import requests

def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']


tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature for provided coordinates in celsius.",
        "parameters": {
            "type": "object",
            "properties": {
                "latitude": {
                    "type": "number",
                    "description": "Latitude of a place"
                },
                "longitude": {
                    "type": "number",
                    "description": "Longitude of a place"
                }
            },
            "required": [
                "latitude", "longitude"
            ],
            "additionalProperties": False
        },
        "strict": True
    }
}]

messages = [{"role": "user", "content": "What is the weather like in Paris today?"}]
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools
)

##### Step 2: Model decides to call function(s) – model returns the name and input arguments.

In [25]:
completion.choices[0].message.tool_calls[0].to_dict()

{'id': 'call_6g4EYMNteZRPD5JQWG8mMRmg',
 'function': {'arguments': '{"latitude":48.8566,"longitude":2.3522}',
  'name': 'get_weather'},
 'type': 'function'}

##### Step 3: Execute function code – parse the model's response and handle function calls.

In [26]:
import json
tool_call = completion.choices[0].message.tool_calls[0]
args = json.loads(tool_call.function.arguments)
result = get_weather(**args)
result

6.0

##### Step 4: Supply model with results – so it can incorporate them into its final response.

In [27]:
messages.append(completion.choices[0].message)  # append model's function call message
messages.append({                               # append result message
    "role": "tool",
    "tool_call_id": tool_call.id,
    "content": str(result)
})
completion_2 = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)

##### Model responds – incorporating the result in its output.

In [28]:
completion_2.choices[0].message.content

'The current temperature in Paris is 6°C.'

### Defining the functions

A function is defined by its schema, which informs the model what it does and what input arguments it expects. It comprises the following fields:

1. name: The function's name
2. description: Details on when and how to use this function
3. parameters: JSON Schema defining the function's input arguments
4. strict: Whether to enable strict schema adherence when generating the function call. If set to true, the model will follow the exact schema defined in the parameters field. 

### Best Practices for defining functions
1. Write clear and detailed function names, parameter descriptions, and instructions.
2. Offload the burden from the model and use code where possible.
    - Don't make the model fill arguments you already know
    - Combine functions that are always called in sequence
3. Keep the number of functions small for higher accuracy.
    - Aim for fewer than 20 functions at any one time, though this is just a soft suggestion. 

### Additional Configuration

1. Tool Choice: By default the model will determine when and how many tools to use. You can force specific behavior with the tool_choice parameter.
    - Auto (Default): Call zero, one, or multiple functions. tool_choice: "auto"
    - Required: Call one or more functions. tool_choice: "required"
    - Forced Function: Call exactly one specific function. tool_choice: {"type": "function", "function": {"name": "get_weather"}}
    - None: To imitate the behavior of passing no functions

2. parallel_tool_calls: The model may choose to call multiple functions in a single turn. You can prevent this by setting parallel_tool_calls to false, which ensures exactly zero or one tool is called.